In [4]:
# debug 3-1, considering effect of job finding rate in emp group
import random
import math
import itertools # join multiple lists
import matplotlib.pyplot as plt

# Parameters
initial_population = 1000
simulation_years = 50

birth_rate = 0.011 # assume only population 15 years and over reproduce 

death_rate = 0.01 # assume death only happens after retired with count_death = count_senior * death_rate
death_age = 80 #a simple alternative way for the senior is assume they die after 80 years old
retirement_age = 65

# Assumption 3: all population aging from 15 to 64 are in labor force and 
# participation_rate = count 15-64 / total population
initial_participation_rate = 0.7 

initial_aging_rate = 0.1 # aging_rate = count 65 and over/ total population
initial_unemployment_rate = 0.05

job_finding_rate = 0.563
job_separation_rate = 0.47

class Person:
    def __init__(self, age, employed):
        self.age = age
        self.employed = employed
    
    def update_age(self):
        self.age += 1
        
    def is_newborn(self):
        return self.age == 0
    
    def is_young(self):
        return self.age < 15
    
    def is_retired(self):
        return self.age >= retirement_age
    
    def is_dead(self):
        return self.age > 80
    
    def is_unemployed(self):
        return not self.employed
    
    def is_employed(self):
        return self.employed

# Initialize workforce data
# Assumption 3: all agents in between 15-64 choose to work, that is, labor force participation is 100%
# then unemployment population + employment population = workforce
initial_workforce = int(initial_population * initial_participation_rate) # 700
initial_senior = int(initial_population * initial_aging_rate) # 100
initial_unemp = int(initial_workforce * initial_unemployment_rate) # 700*0.05=35

# Initialize population
population = []
dependent = []
senior = []
young = []
unemp = []
emp = []
unem_rate = [initial_unemployment_rate]

random.seed(1)
# the cohort of dependent
for human in range(initial_population - initial_workforce):
    person = Person(age=random.randint(*random.choice([(0, 14), (65, 80)])),employed=False)
    senior.append(person) if person.age >= 65 else young.append(person)

# the cohort of unemployment    
for human in range(initial_unemp):
    person = Person(age=random.randint(15, 64), employed=False)
    unemp.append(person)

# the cohort of employment
for _ in range(initial_workforce-initial_unemp):
    person = Person(age=random.randint(15, 64), employed=True)
    emp.append(person)

workforce = list(itertools.chain(unemp,emp))
population = list(itertools.chain(senior, young, unemp, emp))

#print('senior, young, unemp, emp: ', [len(senior), len(young),len(unemp),len(emp)])

# debug 2-2

# initial values: retirees, newborn, unemp, emp:  [152, 148, 35, 665]

#class change( ):
#  age-related transition from young to adult, from workforce to retired, from retired to death 

'''
def __init__(self, group, change_rate):
    self.group = group
    self.change_rate = change_rate
'''
def group_out(age, group_out):
    num1 = 0
    for person in group_out:
        if person.age == age:
            num1 += 1
            group_out.remove(person)
            
    return [num1, group_out]

def group_in(age, group_in):
    num2 = 0
    for person in group_in:
        if person.age == age:
            num2 += 1
            group_in.append(person)
            
    return [num2, group_in]

# change_rate related transition     
# transition from unemp to emp (job finding rate), from emp to unemp (job separation rate)
# from retired to death (death rate), from newborn to young (birth rate) 
# delete population from the group each year
def del_pop(group, change_rate):
    for _ in range(math.ceil(float(len(group))*change_rate)):
        random_element = random.choice(group)
        group.remove(random_element)
        
    return group
            
# adding population to the group each year        
def add_pop(group, change_rate):
    for _ in range(math.ceil(float(len(group))*change_rate)):
        random_element = random.choice(group)
        group.append(random_element)
        
    return group
    
# Simulation loop
#cohorts = [retired, newborn, unemployed, employment]
cohorts = [len(senior)], [len(young)], [len(unemp)], [len(emp)]
n = simulation_years - 40
for year in range(1, n):
    new_population = []
    
    young_this_year = 0
    retired_this_year = 0
    unemployed_this_year = 0
    new_employment = 0
    
    for person in population:
        person.update_age()

        if person.is_retired():
            if not person.is_dead():
                retired_this_year += 1
                person.employed = False
            else:
                del_pop(senior, death_rate)
                    
        if person.is_young():
            young_this_year += 1
            
        if 14 < person.age < 65 and person.is_unemployed():
            if random.random() < initial_unemployment_rate:
                person.employed = True
            else:
                unemployed_this_year += 1
                        
        if 14 < person.age < 65 and person.is_employed():
            new_employment += 1
        
    retired_this_year -= math.ceil(retired_this_year * death_rate)
    young_this_year += math.ceil((retired_this_year + unemployed_this_year + new_employment) * birth_rate)
    
    my_list = [retired_this_year, young_this_year, unemployed_this_year, new_employment]
    [cohorts[i].append(my_list[i]) for i in range(len(cohorts))]
    
print('retirees, young, unemp, emp: ', [cohorts[0], cohorts[1], cohorts[2], cohorts[3]])

retirees, young, unemp, emp:  [[152, 149, 155, 164, 168, 180, 186, 188, 193, 197], [148, 149, 141, 129, 121, 113, 104, 93, 82, 70], [35, 38, 44, 55, 61, 65, 66, 72, 79, 86], [665, 662, 653, 633, 623, 608, 604, 598, 588, 579]]
